In [1]:
#from revChatGPT.V2 import Chatbot
#from revChatGPT.V2 import Conversation
#from revChatGPT.V2 import Conversations
import pandas as pd
import requests
from tqdm import tqdm
import json
import time

C:\Users\isen1\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
 # ! pip install openai==0

### Huggingface Serverless inference

In [3]:
# ! pip install huggingface_hub==0.25.2

In [4]:
from huggingface_hub import InferenceClient
import time

In [55]:
with open('hf_key.json', 'r') as infile: key = json.load(infile)
client = InferenceClient(api_key=key['hf']) # insert key here

In [57]:
for message in client.chat_completion(
	model="meta-llama/Llama-3.1-70B-Instruct",
	messages=[{"role": "user", "content": "What is the capital of France?"}],
	max_tokens=500,
	stream=True,
):
    print(message.choices[0].delta.content, end="")

content = "comment: Fuck you, you're my bitch"

prompt = "Consider you read the above comment on Reddit. How offensive do you think it is, on a scale from 1 (not offensive at all) to 5 (very offensive)? Valid options: 1, 2, 3, 4, 5. Begin your response by selecting one of the valid options, then provide a concise explanation for your rating."

response_format = {
    "type": "json",
    "value": {
        "properties": {
            "rating": {"type": "string"},
            "explanation": {"type": "string"},
        },
        "required": ["label", "explanation"],
    },
}

response = client.chat_completion(
	model="meta-llama/Llama-3.1-70B-Instruct",
	messages=[{"role": "user", "content": content + ' ' + prompt}],
  # response_format=response_format,
  stream=False
	# max_tokens=500,
	)

response['choices']#[0]['message']['content']#.split('\n\n')

The capital of France is Paris.

[{'finish_reason': 'stop',
  'index': 0,
  'message': {'role': 'assistant',
   'content': '5\n\nI would rate this comment a 5, as it contains a profanity and a derogatory slur that is dehumanizing and demeaning. The language used is extremely aggressive and misogynistic, implying ownership and control over someone else, which is highly offensive and unacceptable.',
   'tool_calls': None},
  'logprobs': None}]

## Prompt Section

In [10]:
## original prompt
def create_prompt_definition_sexism(definition):

    prompt = f"Your task is to annotate if a piece of text is sexist or not.\
    Later, you get a detailed definition of sexism. \
    Definition of sexism: {definition} "
    
    return prompt

In [11]:
test_text1 = "women are capable of commentating on football matches" 
test_text2 = "women are weak and belong in the kitchen"

test_definition = "sexism is when men comment on women"

In [12]:
prompt = create_prompt_definition_sexism(test_definition)

In [13]:
prompt

'Your task is to annotate if a piece of text is sexist or not.    Later, you get a detailed definition of sexism.     Definition of sexism: sexism is when men comment on women '

#### import definitions, put in dictionary per participant

In [14]:
definitions_data = pd.read_csv("../../data/definitions/Sexism/Definitions_Interaction_ds - Definitions_Interaction_ds.csv")

In [15]:
from collections import defaultdict

def_dict = defaultdict()

for _, row in definitions_data.iterrows():
    participant_dict = defaultdict()
    participant_dict["chosen_definition_10"] = row["chosen_definition_10"]
    participant_dict["def_ChatGPT_1"] = row["def_ChatGPT_1"]
    participant_dict["participant_definition"] = row["participant_define"]
    try: 
        participant_dict["definition_category"] = row["Indira Definition Category"].split(" detail")[0]
    except:
        participant_dict["definition_category"] = "unk"
    def_dict[row["ResponseId"]] = participant_dict

#### add definition prompts to dict

In [16]:
for participant in def_dict.keys():
    def_dict[participant]["prompt_definition_chosen_10"] = create_prompt_definition_sexism(def_dict[participant]["chosen_definition_10"])
    def_dict[participant]["prompt_definition_ChatGPT_1"] = create_prompt_definition_sexism(def_dict[participant]["def_ChatGPT_1"])
    def_dict[participant]["prompt_definition_participant"] = create_prompt_definition_sexism(def_dict[participant]["participant_definition"])


In [17]:
def_dict

defaultdict(None,
            {'EXP_1': defaultdict(None,
                         {'chosen_definition_10': "Sexism refers to prejudice, discrimination, or stereotyping based on a person's sex or gender, typically with a bias against one gender over the other. It can manifest in various forms, including unequal treatment, unfair expectations, and harmful attitudes or beliefs about individuals based on their sex. Examples of everyday sexism in the workplace could include paying women less than men for the same job or assuming certain roles are only suitable for one gender. Online sexism could involve making derogatory comments based on gender or perpetuating harmful stereotypes in digital spaces. ",
                          'def_ChatGPT_1': "Certainly! Sexism refers to prejudice, discrimination, or stereotyping based on a person's sex or gender, typically with a bias against one gender over the other. Sexism can manifest in various forms, including unequal treatment, unfair expectation

#### Add texts to prompt

In [18]:
def add_texts_to_prompts(prompt_dict, prompt_version, text):
    preamble = prompt_dict[prompt_version]
    prompt = preamble + f"With this definition in mind and a piece of text, please reply with either of the three options and nothing else: 1) sexist, 2) non-sexist, 3) can’t say. Please use the last option as little as possible and if you do, provide a one sentence explanation. Do not give an explanation if you pick ’sexist’ or ’non-sexist’. \
    We are ready to begin now. Is this sentence sexist based on the past definition’ : {text} - ’Please reply with either of the three options and nothing else: 1) sexist, 2) non-sexist, 3) can’t say."
    return prompt

In [19]:
# def_dict

### Classification test 

In [20]:
prompt = add_texts_to_prompts(participant_dict, 
                             "prompt_definition_participant", 
                            "Girls are dumber than boys")

response = client.chat_completion(
	model="meta-llama/Llama-3.1-70B-Instruct",
	messages=[{"role": "user", "content": prompt}],
  # response_format=response_format,
  stream=False
	# max_tokens=500,
	)

In [21]:
response['choices']#[0]['message']['content']#.split('\n\n')

[{'finish_reason': 'stop',
  'index': 0,
  'message': {'role': 'assistant', 'content': '1) sexist', 'tool_calls': None},
  'logprobs': None}]

In [ ]:
all_results_dict_of_dfs_LLaMa.pickle

### Data to classify

In [22]:
test_sets = ['in_domain', 'out_of_domain', 
             'out_of_domain_2', 'out_of_domain_3',# 'out_of_domain_4', 
             'hatecheck'
]

In [23]:
construct_names = ['sexism']

construct = 'sexism'
pos_label = 'sexist'
neg_label = 'non-sexist'

In [36]:
test_set_data = {}

#test with 10, remove head(10) to test on all data

for construct in construct_names:
    test_set_data[construct] = {}
    for test_set in test_sets:
        test_path = '../../data/data/%s/test/%s.csv' %(construct, test_set)
        test_set_data[construct][test_set] = pd.read_csv(test_path, sep = '\t').sample(n=500, random_state=123).head(10)

In [37]:
test_set_data["sexism"]

{'in_domain':                            _id  \
 701   5d654927ce2c6bd733479677   
 186   5d654920ce2c6bd733478932   
 13    5d654922ce2c6bd733478daa   
 1134  5d65490ece2c6bd733476c25   
 1165  5d65491ace2c6bd733477eda   
 45    5d65491ace2c6bd733477e64   
 643   5d654924ce2c6bd73347955d   
 1177  5d6eafdf0fde4e38fac33a1e   
 1084  5d654923ce2c6bd7334792ae   
 709   5d654921ce2c6bd733478bfe   
 
                                                    text      sexism  
 701   don’t they all look the same without makeup 💄?...  non-sexist  
 186   I really don't think Brides Magazine belongs i...  non-sexist  
 13    99% of the women I reach out to as sources ben...  non-sexist  
 1134  Guy whose car we blocked in high-fiving marche...  non-sexist  
 1165  When her hands get sore, she finds another way...  non-sexist  
 45                @pondereroftruth @sara771987 Perhaps.  non-sexist  
 643   @RikSennin @lizzyf620 you actually believe tha...  non-sexist  
 1177  I accept that in my circl

In [38]:
sample_data = pd.read_csv("../../data/data/sexism/train/data_sample.tsv",  sep = '\t')

In [39]:
sample_data

,_id,text,sexism
0,5d654925ce2c6bd7334795bd,i can't take girls serious when talk about sports,sexist
1,5d654901ce2c6bd733475c03,#womensday Women have surely outdone men if t...,sexist
2,5d65490ace2c6bd7334763c0,You've undercooked eggs and probably poisoned ...,non-sexist
3,5ddc4fe8174b03a4c6aaf5d5,A daughter in a family should have the same pr...,non-sexist
4,5d654921ce2c6bd733478ac5,a womans place IS the kitchen,sexist
5,5d654915ce2c6bd73347758e,"@dleemand ""can't take compliments"" = ""don't wa...",non-sexist
6,5d654922ce2c6bd733478fbb,I think it's cool having women working at #hom...,non-sexist
7,5d654921ce2c6bd733478c8c,girls sports are just not good,sexist
8,5d65491fce2c6bd733478722,Not sexist...But I cant have a woman discussin...,sexist
9,5d654904ce2c6bd733475d13,ppl always discredit Mingyus height but im tal...,sexist


## Classification w LLaMa models

#### Step 1, do participant definitions, LLaMa

In [29]:
from tqdm import tqdm

In [31]:
# for each test set
results_participantdef_GPT4o = {}

for domain, test_set in test_set_data["sexism"].items():
    # we have a result dictionary of dictionaries per participant
    result_dict = defaultdict()
    for participant, participant_dict in def_dict.items():
            part_dict = defaultdict()
            for _, row in tqdm(test_set_data["sexism"][domain].iterrows(),
                               total=test_set_data["sexism"][domain].shape[0]):
                prompt = add_texts_to_prompts(participant_dict, 
                                          "prompt_definition_participant", 
                                          row["text"])
                try:
                    responses = client.chat_completion(
                        model="meta-llama/Llama-3.1-70B-Instruct",
                        messages=[{"role": "user", "content": prompt}],
                        # response_format=response_format,
                        stream=False)
                    
#                     responses = client.chat.completions.create(model=GPT_4o,
#                                                                messages=[{"role": "user", 
#                                                                       "content": prompt}],
#                                                                         temperature=0,
#                                                                          max_tokens=200,
#                                                                          top_p=1,
#                                                                          frequency_penalty=0,
#                                                                          presence_penalty=0)                                                                    
                except:
                    time.sleep(60)
                    responses = client.chat_completion(
                        model="meta-llama/Llama-3.1-70B-Instruct",
                        messages=[{"role": "user", "content": prompt}],
                        # response_format=response_format,
                        stream=False)
                part_dict[row["_id"]] = responses.choices[0].message.content                                                          
            results_participantdef_GPT4o[participant] = part_dict 
            result_dict['LlaMa3_70b'] = results_participantdef_GPT4o
    test_set_data['sexism'][domain]["results"] = str(result_dict)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.01s/it]


In [32]:
with open("results_500items_llama.txt", "w+") as f:
    f.write(str(result_dict))

In [44]:
result_dict["LlaMa3_70b"]['EXP_2']

defaultdict(None,
            {36: '1) sexist',
             2009: '1) sexist',
             596: '1) sexist',
             1974: '1) sexist',
             2317: '2) non-sexist',
             3834: '1) sexist',
             1046: '2) non-sexist',
             3306: '1) sexist',
             2791: 'non-sexist',
             337: '1) sexist'})

In [49]:
result_dict["LlaMa3_70b"]['EXP_2']

defaultdict(None,
            {36: '1) sexist',
             2009: '1) sexist',
             596: '1) sexist',
             1974: '1) sexist',
             2317: '2) non-sexist',
             3834: '1) sexist',
             1046: '2) non-sexist',
             3306: '1) sexist',
             2791: 'non-sexist',
             337: '1) sexist'})

In [51]:
from sklearn.metrics import classification_report

# change the label extraction because it seems to be different for llama

for participant in result_dict["LlaMa3_70b"].keys():
    print(participant)
    labels = [i.split(")")[1].strip().split(".")[0] if len(i.split(")")) > 1 else i\
              for i in result_dict["LlaMa3_70b"][participant].values()]
    labels = [i if i in (['sexist', 'non-sexist']) else 'non-sexist' for i in labels] # defaults to 'non-sexist'
#     print(labels)
    
    truth = test_set_data["sexism"]["in_domain"]["sexism"].tolist()
    print(classification_report(truth, labels))
    print()

EXP_1
              precision    recall  f1-score   support

  non-sexist       0.67      0.25      0.36         8
      sexist       0.14      0.50      0.22         2

    accuracy                           0.30        10
   macro avg       0.40      0.38      0.29        10
weighted avg       0.56      0.30      0.34        10


EXP_2
              precision    recall  f1-score   support

  non-sexist       0.67      0.25      0.36         8
      sexist       0.14      0.50      0.22         2

    accuracy                           0.30        10
   macro avg       0.40      0.38      0.29        10
weighted avg       0.56      0.30      0.34        10


EXP_3
              precision    recall  f1-score   support

  non-sexist       0.67      0.25      0.36         8
      sexist       0.14      0.50      0.22         2

    accuracy                           0.30        10
   macro avg       0.40      0.38      0.29        10
weighted avg       0.56      0.30      0.34        10


In [52]:
test_set_data["sexism"][domain]

,_id,text,sexism
5,36,Disgusted by all the women around here.,sexist
277,2009,What if we just killed all women?,sexist
85,596,I'm going to stab a woman tomorrow.,sexist
272,1974,Isn't it typical of a woman to be that stupid?,sexist
318,2317,There are very few women at my school.,non-sexist
498,3834,You are nothing but a b1tch.,sexist
148,1046,I keep two garden hoes out in my tool shed.,non-sexist
425,3306,Womn belong in a zoo.,sexist
384,2791,It hurts to see all these people saying female...,non-sexist
48,337,Women are lesser beings.,sexist


### Step 2, do GPT 1 definitions

In [53]:
# for each test set
for domain, test_set in test_set_data["sexism"].items():
    # we have a result dictionary of dictionaries per participant
    result_dict = defaultdict()
    for participant, participant_dict in def_dict.items():
            part_dict = defaultdict()
            for _, row in test_set_data["sexism"][domain].iterrows():
                prompt = add_texts_to_prompts(participant_dict, 
                                          "prompt_definition_ChatGPT_1", 
                                          row["text"])
                try:
                    responses = client.chat_completion(
                        model="meta-llama/Llama-3.1-70B-Instruct",
                        messages=[{"role": "user", "content": prompt}],
                        # response_format=response_format,
                        stream=False)                                                                    
                except:
                    time.sleep(60)
                    responses = client.chat_completion(
                        model="meta-llama/Llama-3.1-70B-Instruct",
                        messages=[{"role": "user", "content": prompt}],
                        # response_format=response_format,
                        stream=False) 
                part_dict[row["_id"]] = responses.choices[0].message.content                                                          
            results_participantdef_GPT4o[participant] = part_dict 
            result_dict[domain] = results_participantdef_GPT4o
    test_set_data['sexism'][domain]["results_llamadef1"] = str(result_dict)

In [54]:
with open("F1_500items_llamadef1.txt", "w+") as f:
    f.write(result_dict)

TypeError: write() argument must be str, not collections.defaultdict

In [ ]:
from sklearn.metrics import classification_report

for participant in result_dict["GPT4o"].keys():
    print(participant)
    labels = [i.split(")")[1].strip().split(".")[0] for i in result_dict["GPT4o"][participant].values()]
#     print(labels)
    truth = test_set_data["sexism"]["in_domain"]["sexism"].tolist()
    print(classification_report(truth, labels))
    print()

### Step 3 do GPT 10 definitions

In [ ]:
# for each test set
for domain, test_set in test_set_data["sexism"].items():
    # we have a result dictionary of dictionaries per participant
    result_dict = defaultdict()
    for participant, participant_dict in def_dict.items():
            part_dict = defaultdict()
            for _, row in test_set_data["sexism"][domain].iterrows():
                prompt = add_texts_to_prompts(participant_dict, 
                                          "prompt_definition_chosen_10", 
                                          row["text"])
                try:
                    responses = client.chat.completions.create(model=GPT_4o,
                                                               messages=[{"role": "user", 
                                                                      "content": prompt}],
                                                                        temperature=0,
                                                                         max_tokens=200,
                                                                         top_p=1,
                                                                         frequency_penalty=0,
                                                                         presence_penalty=0)                                                                    
                except:
                    time.sleep(60)
                    responses = client.chat.completions.create(model=GPT_4o,
                                                               messages=[{"role": "user", 
                                                                      "content": prompt}],
                                                                        temperature=0,
                                                                         max_tokens=200,
                                                                         top_p=1,
                                                                         frequency_penalty=0,
                                                                         presence_penalty=0) 
                part_dict[row["_id"]] = responses.choices[0].message.content                                                          
            results_participantdef_GPT4o[participant] = part_dict 
            result_dict[domain] = results_participantdef_GPT4o
    test_set_data['sexism'][domain]["results_GPTdef10"] = str(result_dict)

In [ ]:
with open("F1_500items_GPTdef10.txt", "w+") as f:
    f.write(result_dict)

In [ ]:
from sklearn.metrics import classification_report

for participant in result_dict["GPT4o"].keys():
    print(participant)
    labels = [i.split(")")[1].strip().split(".")[0] for i in result_dict["GPT4o"][participant].values()]
#     print(labels)
    truth = test_set_data["sexism"]["in_domain"]["sexism"].tolist()
    print(classification_report(truth, labels))
    print()

### Step 4, do F1 scores

### Step 5, visualize F1 scores 